In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd

import glob
from sunpy.coordinates import sun
from astropy.utils.data import download_file
import astropy.units as u
import astropy.io.fits as fits
import astropy.constants as astro_const
from astropy.coordinates import SkyCoord
import os
import sunpy
import numpy as np
import gzip
import requests
import urllib.request
from bs4 import BeautifulSoup
from datetime import datetime
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from sunpy.map import Map
from sunpy.sun import constants as const
from sunpy.coordinates import frames
from sunpy.coordinates import sun
import pfsspy
import pfsspy.tracing as tracing
from sunkit_instruments import suvi
import matplotlib.colors as colors
from astropy.visualization import ImageNormalize, LogStretch, LogStretch, PercentileInterval
from tqdm import tqdm

# Construct the full path to the scripts directory
script_path = os.path.abspath('./scripts')

# Add the scripts directory to sys.path
import sys
if script_path not in sys.path:
    sys.path.append(script_path)

import fwdtools as fwd
import solar_data_utils as myfuncs
# Reload to update with new changes
import importlib
importlib.reload(fwd)
importlib.reload(myfuncs)

plt.rcParams['figure.figsize'] = [7, 7]
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['savefig.format'] = 'png'
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['savefig.facecolor'] = 'white'

# set the directory path where the data is importe/stored
data_dir = '/home/mnedal/data'
basedir = '/home/mnedal/data/'
savedir = '/home/mnedal/repos/dias_work'

Select the date and time of the observation 

In [ ]:
YEAR = '2024'
MONTH = '05'
DAY = '14'
HOUR = '17'
MINUTE = '40'

# construct the datetime object 
mydatetime = datetime(int(YEAR), int(MONTH), int(DAY), int(HOUR), int(MINUTE))

# get the Carrington rotation number at the specified datetime 
cr = str(int(sun.carrington_rotation_number(t=mydatetime)))
print(cr)

# set the field of view of LOFAR observations 
FOV = 3750

In [ ]:
def find_links(url='http://www.python.org'):
    """
    Get all the links in a webpage. 
    Source: https://stackoverflow.com/questions/20150184/make-a-list-of-all-the-files-on-a-website 
    """
    soup = BeautifulSoup(requests.get(url).text)
    hrefs = []
    for a in soup.find_all('a'):
        hrefs.append(a['href'])
    return hrefs

In [ ]:
# show the links of the GONG files 
gong_url = 'https://gong2.nso.edu/oQR/zqs/%s%s/mrzqs%s%s%s/' %(YEAR, MONTH, YEAR[-2:], MONTH, DAY)
list_gong_links = find_links(url=gong_url)

# show what you've found 
for i, link in enumerate(list_gong_links):
    print(i, link)

Choose which file to download 

In [ ]:
gong_file_index = 21
gong_filename = list_gong_links[gong_file_index]

print(basedir + gong_filename)

Load a GONG magnetic field map at the specified datetime 

In [ ]:
if not os.path.exists(basedir + gong_filename[:-3]) and not os.path.exists(basedir + gong_filename):
    urllib.request.urlretrieve(url=gong_url+gong_filename, filename=basedir+gong_filename)

if not os.path.exists(basedir + gong_filename[:-3]):
    with gzip.open(basedir + gong_filename, 'rb') as f:
        with open(basedir + gong_filename[:-3], 'wb') as g:
            g.write(f.read())

In [ ]:
# make a sunpy map of the file 
gong_map = Map(basedir + gong_filename[:-3])

# fix a bug in the GONG map file
if 'cunit1' not in gong_map.meta:
    gong_map.meta['cunit1'] = u.deg

Load the corresponding SUVI 195$\\A^o$ map 

In [ ]:
def load_suvi(start=None, end=None, channel=195):
    """
    * 9.4 nm (FeXVIII)
    * 13.1 nm (FeXXI)
    * 17.1 nm (FeIX/X)
    * 19.5 nm (FeXII)
    * 28.4 nm (FeXV)
    * 30.4 nm (HeII)
    """
    dt_dict = split_datetime(start=start, end=end)
    data_path = f"{data_dir}/SUVI/{dt_dict['start_year']}{dt_dict['start_month']}{dt_dict['start_day']}/{channel}A"
    root_filename = f"dr_suvi-l2-ci{channel}_g18_s"
    data = sorted(glob.glob(f"{data_path}/{root_filename}{dt_dict['start_year']}{dt_dict['start_month']}{dt_dict['start_day']}*.fits"))
    
    start_file_to_find = f"{data_path}/{root_filename}{dt_dict['start_year']}{dt_dict['start_month']}{dt_dict['start_day']}T{dt_dict['start_hour']}0000Z_e{dt_dict['start_year']}{dt_dict['start_month']}{dt_dict['start_day']}T{dt_dict['start_hour']}0400Z_v1-0-2.fits"
    end_file_to_find   = f"{data_path}/{root_filename}{dt_dict['end_year']}{dt_dict['end_month']}{dt_dict['end_day']}T{dt_dict['end_hour']}2800Z_e{dt_dict['end_year']}{dt_dict['end_month']}{dt_dict['end_day']}T{dt_dict['end_hour']}3200Z_v1-0-2.fits"
    
    idx1 = data.index(start_file_to_find)
    idx2 = data.index(end_file_to_find)
    
    chosen_files = data[idx1:idx2]
    
    map_objects = []
    for i, file in enumerate(chosen_files):
        m = suvi.files_to_map(file, despike_l1b=True)
        min_range = 0
        if channel == 94:
            max_range = 20
        elif channel == 171:
            max_range = 20
        elif channel == 131:
            max_range = 20
        elif channel == 195:
            max_range = 50
        elif channel == 284:
            max_range = 50
        elif channel == 304:
            max_range = 100
        
        m.plot_settings['norm'] = ImageNormalize(vmin=min_range, vmax=max_range, stretch=LogStretch())
        map_objects.append(m)
        print(f'SUVI image {i} is done')
    return map_objects



def load_lasco(start=None, end=None, detector='C2'):
    """
    Load SOHO/LASCO C2 or C3 images as sunpy maps.
    """
    dt_dict = split_datetime(start=start, end=end)
    data = sorted(glob.glob(f"{data_dir}/LASCO_{detector}/LASCO_{detector}_{dt_dict['start_year']}{dt_dict['start_month']}{dt_dict['start_day']}*.jp2"))
    
    start_file_to_find = f"{data_dir}/LASCO_{detector}/LASCO_{detector}_{dt_dict['end_year']}{dt_dict['start_month']}{dt_dict['start_day']}T{dt_dict['start_hour']}{dt_dict['start_minute']}.jp2"
    end_file_to_find = f"{data_dir}/LASCO_{detector}/LASCO_{detector}_{dt_dict['end_year']}{dt_dict['end_month']}{dt_dict['end_day']}T{dt_dict['end_hour']}{dt_dict['end_minute']}.jp2"
    
    idx1 = data.index(start_file_to_find)
    idx2 = data.index(end_file_to_find)
    chosen_files = data[idx1:idx2]
    
    map_objects = []
    for i, file in enumerate(chosen_files):
        m = sunpy.map.Map(file)
        m.meta['bunit'] = 'ct' # a workaround for C2 and C3 jp2 images
        m.plot_settings['norm'] = ImageNormalize(vmin=0, vmax=250)
        map_objects.append(m)
        print(f'LASCO {detector} image {i} is done')
    return map_objects



def split_datetime(start=None, end=None):
    
    START_DATE, START_TIME = start.split('T')
    END_DATE, END_TIME = end.split('T')

    START_YEAR, START_MONTH, START_DAY = START_DATE.split('-')
    END_YEAR, END_MONTH, END_DAY = END_DATE.split('-')

    START_HOUR, START_MINUTE, START_SECOND = START_TIME.split(':')
    END_HOUR, END_MINUTE, END_SECOND = END_TIME.split(':')

    datetime_dict = {
        'start_year': START_YEAR,
        'start_month': START_MONTH,
        'start_day': START_DAY,
        'start_hour': START_HOUR,
        'start_minute': START_MINUTE,
        'start_second': START_SECOND,
        
        'end_year': END_YEAR,
        'end_month': END_MONTH,
        'end_day': END_DAY,
        'end_hour': END_HOUR,
        'end_minute': END_MINUTE,
        'end_second': END_SECOND
    }
    return datetime_dict




def remove_redundant_maps(maps):
    """
    Remove redundant SunPy maps, keeping only one map per unique timestamp.

    Parameters:
    maps (list): List of SunPy Map objects. Each map is expected to have a 'date-obs' 
                 key in its metadata that provides the observation timestamp.

    Returns:
    list: A list of unique SunPy Map objects, one per unique timestamp.
    
    Example:
    >>> unique_maps = remove_redundant_maps(list_of_sunpy_maps)
    """
    unique_maps = {}
    for m in maps:
        timestamp = m.latex_name
        if timestamp not in unique_maps:
            unique_maps[timestamp] = m
    return list(unique_maps.values())



def apply_runratio(maps):
    """
    Apply running-ratio image technique on EUV images.
    See: https://iopscience.iop.org/article/10.1088/0004-637X/750/2/134/pdf
        Inputs: list of EUV sunpy maps.
        Output: sequence of run-ratio sunpy maps.
    """
    runratio = [m / prev_m.quantity for m, prev_m in zip(maps[1:], maps[:-1])]
    m_seq_runratio = sunpy.map.Map(runratio, sequence=True)
    
    for m in m_seq_runratio:
        m.data[np.isnan(m.data)] = 1
        m.plot_settings['norm'] = colors.Normalize(vmin=0, vmax=2)
        m.plot_settings['cmap'] = 'Greys_r'
    
    return m_seq_runratio



def enhance_contrast(image, vmin, vmax):
    """
    Enhance contrast by clipping and normalization.
    """
    image_clipped = np.clip(image, vmin, vmax)
    image_normalized = (image_clipped - vmin) / (vmax - vmin)
    return image_normalized



def calculate_percentiles(image, lower_percentile=3, upper_percentile=97):
    """
    Calculate vmin and vmax based on the 1st and 99th percentiles.
    """
    vmin = np.percentile(image, lower_percentile)
    vmax = np.percentile(image, upper_percentile)
    return vmin, vmax





def geo_to_cartesian(latitude, longitude, radius=1):
    """
    Convert geographic coordinates to Cartesian coordinates.
    """
    # Convert degrees to radians
    lat_rad = np.deg2rad(latitude)
    lon_rad = np.deg2rad(longitude)
    
    # Cartesian coordinates
    x = radius * np.cos(lat_rad) * np.cos(lon_rad)
    y = radius * np.cos(lat_rad) * np.sin(lon_rad)
    z = radius * np.sin(lat_rad)
    
    return x, y, z

In [ ]:
suvi_map_objects = load_suvi(start='2024-05-14T17:00:00', end='2024-05-14T18:00:00', channel=195)

In [ ]:
clean_suvi_maps = remove_redundant_maps(suvi_map_objects)
suvi_seq_runratio = apply_runratio(clean_suvi_maps)

In [ ]:
index = 7
m = suvi_seq_runratio[index-1]
m.plot_settings['norm'] = colors.Normalize(vmin=0.7, vmax=1.3)
suvi_map = clean_suvi_maps[index]

fig = plt.figure(figsize=[15,7])

ax = fig.add_subplot(121, projection=suvi_map)
suvi_map.plot(axes=ax)
ax.grid(False)
# ax.set_xlim(left=0, right=400)
# ax.set_ylim(bottom=600, top=1000)

ax = fig.add_subplot(122, projection=m)
m.plot(axes=ax, cmap='Greys_r')
# ax.set_xlim(left=0, right=400)
# ax.set_ylim(bottom=600, top=1000)

fig.tight_layout()
plt.show()

In [ ]:
suvi_map = clean_suvi_maps[index]
suvi_map.plot_settings['norm'] = colors.Normalize(vmin=0, vmax=15)

fig = plt.figure()
ax = plt.subplot(projection=suvi_map)
img = suvi_map.plot(axes=ax, cmap='gray_r')
plt.colorbar(img)
plt.show()

## The PFSS Model

* The Sun’s magnetic field is complex and highly dynamic, particularly in the corona.
* Near the Sun’s surface, the magnetic field lines are closed, forming loops.
* As you move further from the Sun, these loops open up and the magnetic field becomes more radial.
* The PFSS model helps simplify this complex structure by assuming that at a certain distance from the Sun, called the source surface, the magnetic field transitions to this radial state.
* Below the source surface, the model assumes the magnetic field is potential, meaning it is derived from a scalar potential and has no electric currents (it is curl-free).
* This is an approximation that helps in solving the magnetic field equations more easily.
* The source surface is typically set at a radius of about 2.5 $R_\odot$.
* This value is chosen because it roughly corresponds to where solar wind starts to dominate, stretching the magnetic field lines into a radial configuration.
* The parameter `r`, which is set to 1.2 $R_\odot$ represents the radial distance from the center of the Sun at which you are starting the tracing of magnetic field lines.
* The value 1.2 is commonly chosen because it lies in the lower corona, where the magnetic field structure is still influenced by the solar surface but is starting to show the effects of the solar wind and the transition to more open field structures.
* In the PFSS model, the magnetic field is computed between the Sun’s surface and the source surface (often at 2.5 $R_\odot$).
* By setting `r` to 1.2 $R_\odot$, you are examining the magnetic field lines within this region, but closer to the Sun’s surface rather than at the outer boundary.

The [PFSS](https://pfsspy.readthedocs.io/en/stable/) solution is calculated on a regular 3D grid in (phi, s, rho), where
rho = ln(r), and r is the standard spherical radial coordinate.

We need to define the number of grid points in rho, and the source surface radius.

From the boundary condition, number of radial grid points, and source surface, we now construct an `Input` object that stores this information.

Using the `Input` object, plot the input photospheric magnetic field.

In [ ]:
nrho = 50 # number of rho grid points
rss = 3   # source surface radius
pfss_in = pfsspy.Input(gong_map, nrho, rss)
pfss_out = pfsspy.pfss(pfss_in)

Now we construct a lat x lon grid of footpoitns to trace some magnetic field lines from.

These coordinates are defined in the helioprojective frame of the SUVI image.

In [ ]:
num_footpoints = 20
lat = np.linspace(-0.4*np.pi, 0.5*np.pi, num_footpoints, endpoint=False)
lon = np.linspace(-0.5*np.pi, 1.5*np.pi, num_footpoints+10, endpoint=False)

lat, lon = np.meshgrid(lat, lon, indexing='ij')
lat, lon = lat.ravel()*u.rad, lon.ravel()*u.rad
r = 1.2*const.radius

# Make a 2D grid from these 1D points 
seeds = SkyCoord(lon, lat, r, frame=pfss_out.coordinate_frame)
tracer = tracing.FortranTracer()
flines = tracer.trace(seeds, pfss_out)

In [ ]:
# Map of the footpoints
fig = plt.figure()
ax = plt.subplot(projection=suvi_map)
suvi_map.plot(axes=ax, cmap='gray')
ax.plot_coord(seeds, color='red', marker='o', markersize=4, linewidth=0)
plt.show()

Plot the magnetogram and the seed footpoints. The footpoints are centered around the active region metnioned above.

In [ ]:
m = pfss_in.map
fig = plt.figure()
ax = plt.subplot(projection=m)
gon = m.plot()
ax.plot_coord(seeds, color='black', marker='o', linewidth=0, markersize=2)
fig.colorbar(gon, fraction=0.023, pad=0.03, label=pfss_in.map.meta['bunit'])
plt.show()

In [ ]:
fig = plt.figure()
ax = plt.subplot(111, projection=suvi_map)
im = suvi_map.plot(axes=ax, cmap='gray_r')
# suvi_map.draw_limb()
# suvi_map.draw_grid()

for fline in flines:
    ax.plot_coord(fline.coords, alpha=0.3, color='red', linewidth=1)

axins = inset_axes(ax,
                   width='5%',    # width = 5% of parent_bbox width 
                   height='50%',  # height : 50% 
                   loc='lower left', bbox_to_anchor=(1.02,0,1,1), bbox_transform=ax.transAxes, borderpad=0.01)
fig.colorbar(im, cax=axins)
ax.set_xlabel('Helioprojective Longitude [arcsec]')
ax.set_ylabel('Helioprojective Latitutde [arcsec]')
ax.set_xlim(left=0, right=int(suvi_map.dimensions.x.value))
ax.set_ylim(bottom=0, top=int(suvi_map.dimensions.y.value))
plt.show()

In [ ]:
fig = plt.figure()
ax = plt.subplot(111, projection=suvi_map)
im = suvi_map.plot(axes=ax, cmap='gray_r')
suvi_map.draw_limb(color='orange')

for fline in flines:
    color = {0: 'black', -1: 'tab:blue', 1: 'tab:red'}.get(fline.polarity)
    ax.plot_coord(fline.coords, alpha=0.5, color=color, linewidth=1)

axins = inset_axes(ax,
                   width='5%',    # width = 5% of parent_bbox width 
                   height='50%',  # height : 50% 
                   loc='lower left', bbox_to_anchor=(1.02,0,1,1), bbox_transform=ax.transAxes, borderpad=0.01)
fig.colorbar(im, cax=axins)
# ax.set_xlabel('Helioprojective Longitude [arcsec]')
# ax.set_ylabel('Helioprojective Latitutde [arcsec]')
ax.set_xlim(left=0, right=int(suvi_map.dimensions.x.value))
ax.set_ylim(bottom=0, top=int(suvi_map.dimensions.y.value))
plt.show()

Plot the Running-Ratio GOES/SUVI 195$\\A^o$ with the PFSS Model

In [ ]:
m = suvi_seq_runratio[index-1]
m.plot_settings['norm'] = colors.Normalize(vmin=0.7, vmax=1.3)

fig = plt.figure()
ax = plt.subplot(projection=m)
m.plot(axes=ax, cmap='gray')

with tqdm(total=len(flines), desc='Plotting the field lines') as pbar:
    for fline in flines:
        color = {0:'yellow', -1:'cyan', 1:'red'}.get(fline.polarity)
        ax.plot_coord(fline.coords, alpha=0.5, color=color, linewidth=1)
        pbar.update(1)

ax.set_xlim(left=0, right=int(m.dimensions.x.value))
ax.set_ylim(bottom=0, top=int(m.dimensions.y.value))
fig.tight_layout()
fig.savefig(f'{data_dir}/SUVI_195A_runratio_PFSS.pdf', format='pdf', bbox_inches='tight')
plt.show()

### Plot the GOES/SUVI 195$\\A^o$ with the LASCO C2 image

In [ ]:
lasco_c2_map_objects = load_lasco(start='2024-05-14T17:30:00', end='2024-05-14T20:00:00', detector='C2')

In [ ]:
# remove redundant images
clean_maps_c2 = remove_redundant_maps(lasco_c2_map_objects)

print(f'Before:\nLASCO C2: {len(lasco_c2_map_objects)} images\n')
print(f'After:\nLASCO C2: {len(clean_maps_c2)} images')

In [ ]:
m_c2 = clean_maps_c2[3]

fig = plt.figure()
ax = fig.add_subplot(111, projection=m_c2)
img = m_c2.plot(axes=ax)
m_c2.draw_limb()
ax.grid(False)
plt.colorbar(img, pad=0.02, shrink=0.8)
fig.tight_layout()
plt.show()

In [ ]:
# make run-diff maps
m_seq_runratio_c2 = apply_runratio(clean_maps_c2)

In [ ]:
m_c2_rr = m_seq_runratio_c2[2]

fig = plt.figure()
ax = fig.add_subplot(111, projection=m_c2_rr)
img = m_c2_rr.plot(axes=ax)
m_c2_rr.draw_limb()
ax.grid(False)
plt.colorbar(img, pad=0.02, shrink=0.8)

# ax.set_xlim(left=-120, right=400)
# ax.set_ylim(bottom=400, top=850)
plt.show()

Load the nearest in time GONG map to LASCO C2 time

In [ ]:
gong_file_index = 22
gong_filename = list_gong_links[gong_file_index]

print(basedir + gong_filename)

In [ ]:
if not os.path.exists(basedir + gong_filename[:-3]) and not os.path.exists(basedir + gong_filename):
    urllib.request.urlretrieve(url=gong_url+gong_filename, filename=basedir+gong_filename)

if not os.path.exists(basedir + gong_filename[:-3]):
    with gzip.open(basedir + gong_filename, 'rb') as f:
        with open(basedir + gong_filename[:-3], 'wb') as g:
            g.write(f.read())

In [ ]:
gong_map = Map(basedir + gong_filename[:-3])

if 'cunit1' not in gong_map.meta:
    gong_map.meta['cunit1'] = u.deg

nrho = 50 # umber of rho grid points
rss  = 7  # source surface radius
pfss_in  = pfsspy.Input(gong_map, nrho, rss)
pfss_out = pfsspy.pfss(pfss_in)

num_footpoints = 20
lat = np.linspace(-0.4*np.pi, 0.5*np.pi, num_footpoints, endpoint=False)
lon = np.linspace(-0.5*np.pi, 1.5*np.pi, num_footpoints+10, endpoint=False)

lat, lon = np.meshgrid(lat, lon, indexing='ij')
lat, lon = lat.ravel()*u.rad, lon.ravel()*u.rad
r = 1.2*const.radius

# Make a 2D grid from these 1D points 
seeds  = SkyCoord(lon, lat, r, frame=pfss_out.coordinate_frame)
tracer = tracing.FortranTracer()
flines = tracer.trace(seeds, pfss_out)

In [ ]:
# Map of the footpoints
fig = plt.figure()
ax = fig.add_subplot(111, projection=m_c2_rr)
m_c2_rr.plot(axes=ax)
ax.plot_coord(seeds, color='red', marker='o', markersize=3, linewidth=0)
m_c2_rr.draw_limb()
ax.grid(False)
plt.show()

In [ ]:
lat_ss = np.linspace(-0.4*np.pi, 0.5*np.pi, num_footpoints, endpoint=False)
lon_ss = np.linspace(-0.5*np.pi, 1.5*np.pi, num_footpoints+30, endpoint=False)

lat_ss, lon_ss = np.meshgrid(lat_ss, lon_ss, indexing='ij')
lat_ss, lon_ss = lat_ss.ravel()*u.rad, lon_ss.ravel()*u.rad
ss = SkyCoord(lon_ss, lat_ss, rss*const.radius, frame=pfss_out.coordinate_frame)

In [ ]:
# Map of the footpoints
fig = plt.figure()
ax = fig.add_subplot(111, projection=m_c2_rr)
m_c2_rr.plot(axes=ax)
ax.plot_coord(seeds, color='blue', marker='o', markersize=3, linewidth=0, label='Inner boundary')
ax.plot_coord(ss, color='red', marker='o', markersize=3, linewidth=0, label='Outer boundary')
m_c2_rr.draw_limb()
ax.grid(False)
ax.legend(loc='best')
plt.show()

In [ ]:
m = pfss_in.map

fig = plt.figure()
ax = plt.subplot(projection=m)
gon = m.plot()
ax.plot_coord(seeds, color='black', marker='o', linewidth=0, markersize=2)
fig.colorbar(gon, fraction=0.023, pad=0.03, label=pfss_in.map.meta['bunit'])
plt.show()

In [ ]:
m = pfss_in.map
fig = plt.figure()
ax = plt.subplot(projection=m)
gon = m.plot()
ax.plot_coord(ss, color='black', marker='o', linewidth=0, markersize=2)
fig.colorbar(gon, fraction=0.023, pad=0.03, label=pfss_in.map.meta['bunit'])
plt.show()

In [ ]:
fig = plt.figure()
ax = plt.subplot(111, projection=m_c2_rr)
im = m_c2_rr.plot(axes=ax)

for fline in flines:
    ax.plot_coord(fline.coords, alpha=0.3, color='red', linewidth=1)

axins = inset_axes(ax,
                   width='5%',    # width = 5% of parent_bbox width 
                   height='50%',  # height : 50% 
                   loc='lower left', bbox_to_anchor=(1.02,0,1,1), bbox_transform=ax.transAxes, borderpad=0.01)
fig.colorbar(im, cax=axins)
ax.set_xlim(left=0, right=int(m_c2_rr.dimensions.x.value))
ax.set_ylim(bottom=0, top=int(m_c2_rr.dimensions.y.value))
plt.show()

In [ ]:
fig = plt.figure()
ax = plt.subplot(111, projection=m_c2_rr)
im = m_c2_rr.plot(axes=ax)

with tqdm(total=len(flines), desc='Plotting the field lines') as pbar:
    for fline in flines:
        # color = {0: 'black', -1: 'tab:blue', 1: 'tab:red'}.get(fline.polarity)
        color = {0:'black', -1:'blue', 1:'red'}.get(fline.polarity)
        ax.plot_coord(fline.coords, alpha=0.5, color=color, linewidth=1)
        pbar.update(1)

# axins = inset_axes(ax,
#                    width='5%',    # width = 5% of parent_bbox width 
#                    height='50%',  # height : 50% 
#                    loc='lower left', bbox_to_anchor=(1.02,0,1,1), bbox_transform=ax.transAxes, borderpad=0.01)
# fig.colorbar(im, cax=axins)
ax.set_xlim(left=0, right=int(m_c2_rr.dimensions.x.value))
ax.set_ylim(bottom=0, top=int(m_c2_rr.dimensions.y.value))
fig.tight_layout()
# fig.savefig(f'{data_dir}/LASCO_C2_runratio_PFSS.pdf', format='pdf', bbox_inches='tight')
# fig.savefig(f'{data_dir}/LASCO_C2_runratio_PFSS.png', format='png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
m_c2 = clean_maps_c2[2]

fig = plt.figure()
ax = fig.add_subplot(111, projection=m_c2)
m_c2.plot(axes=ax)
m.draw_limb()

with tqdm(total=len(flines), desc='Plotting the field lines') as pbar:
    for fline in flines:
        ax.plot_coord(fline.coords, alpha=0.5, color='white', linewidth=1)
        pbar.update(1)

ax.grid(False)
ax.set_xlim(left=0, right=int(m_c2.dimensions.x.value))
ax.set_ylim(bottom=0, top=int(m_c2.dimensions.y.value))
fig.tight_layout()
# fig.savefig(f'{data_dir}/LASCO_C2_PFSS.pdf', format='pdf', bbox_inches='tight')
plt.show()

In [ ]:
# show PFSS lines on FORWARD map
file_paths = sorted(glob.glob(f'{data_dir}/FORWARD/*.sav'))

for file in file_paths:
    if 'BMAG' in file:
        bmag = fwd.get_forward_map(file)
    elif 'DENS' in file:
        dens = fwd.get_forward_map(file)

# Calculate the Alfven speed map
mp = astro_const.m_p.to(u.gram).value
valf = bmag.data/np.sqrt(4*np.pi*mp*dens.data)/1e4
VA = sunpy.map.Map(valf, dens.meta)

fig = plt.figure()
ax = fig.add_subplot(111, projection=VA)
img = VA.plot(axes=ax, vmin=400, vmax=1e4,
              title='$V_{alfven}$ [km s$^{-1}$]',
              cmap='Spectral_r')
# VA.draw_limb()
with tqdm(total=len(flines), desc='Plotting the field lines') as pbar:
    for fline in flines:
        ax.plot_coord(fline.coords, alpha=0.4, color='black', linewidth=1)
        pbar.update(1)

ax.grid(False)
ax.set_xlim(left=0, right=int(VA.dimensions.x.value))
ax.set_ylim(bottom=0, top=int(VA.dimensions.y.value))
plt.colorbar(img, ax=ax, shrink=0.7, pad=0.05)
fig.tight_layout()
# fig.savefig(f'{data_dir}/PSIMAS_VA_PFSS.pdf', format='pdf', bbox_inches='tight')
plt.show()

In [ ]:
fig = plt.figure(figsize=[15,5])

ax = fig.add_subplot(131, projection=dens)
img = dens.plot(axes=ax, norm=colors.LogNorm(), title='log(Density [cm$^{-3}$])', cmap='Spectral_r')
plt.colorbar(img, ax=ax, shrink=0.7, pad=0.05)
with tqdm(total=len(flines), desc='Plotting the field lines') as pbar:
    for fline in flines:
        ax.plot_coord(fline.coords, alpha=0.3, color='black', linewidth=1)
        pbar.update(1)
ax.grid(False)
ax.set_xlim(left=0, right=int(dens.dimensions.x.value))
ax.set_ylim(bottom=0, top=int(dens.dimensions.y.value))

ax = fig.add_subplot(132,projection=bmag)
img = bmag.plot(axes=ax, norm=colors.LogNorm(),  title='log(|B| [G])',  cmap='Spectral_r')
plt.colorbar(img, ax=ax, shrink=0.7, pad=0.05)
with tqdm(total=len(flines), desc='Plotting the field lines') as pbar:
    for fline in flines:
        ax.plot_coord(fline.coords, alpha=0.3, color='black', linewidth=1)
        pbar.update(1)
ax.grid(False)
ax.set_xlim(left=0, right=int(bmag.dimensions.x.value))
ax.set_ylim(bottom=0, top=int(bmag.dimensions.y.value))

ax = fig.add_subplot(133, projection=VA)
img = VA.plot(axes=ax, vmin=400, vmax=1e4,  title='$V_{alfven}$ [km s$^{-1}$]', cmap='Spectral_r')
plt.colorbar(img, ax=ax, shrink=0.7, pad=0.05)
with tqdm(total=len(flines), desc='Plotting the field lines') as pbar:
    for fline in flines:
        ax.plot_coord(fline.coords, alpha=0.3, color='black', linewidth=1)
        pbar.update(1)
ax.grid(False)
ax.set_xlim(left=0, right=int(VA.dimensions.x.value))
ax.set_ylim(bottom=0, top=int(VA.dimensions.y.value))

fig.tight_layout()
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d', computed_zorder=False)

theta, phi = np.mgrid[0:np.pi:100j, 0:2*np.pi:100j]
x = np.sin(theta)*np.cos(phi) 
y = np.sin(theta)*np.sin(phi) 
z = np.cos(theta)
ax.plot_surface(x, y, z, color='orange', zorder=0)

with tqdm(total=len(flines), desc='Plotting the field lines') as pbar:
    for n, field_line in enumerate(flines):
        color = {0:'black', -1:'blue', 1:'red'}.get(field_line.polarity)
        coords = field_line.coords
        coords.representation_type = 'cartesian'
        x_line = coords.x/const.radius
        y_line = coords.y/const.radius
        z_line = coords.z/const.radius
        # ax.plot3D(x_line, y_line, z_line, alpha=0.3, color=color, linewidth=1, zorder=10)
        if field_line.polarity==1 or field_line.polarity==-1:
            alpha = 0.3
        else:
            alpha = 0
        ax.plot3D(x_line, y_line, z_line, alpha=alpha, color=color, linewidth=1, zorder=10)
        pbar.update(1)

# draw Sun-Earth LOS direction
x_pos = 3
y_pos = 0
z_pos = 0
x_direct = 1
y_direct = 0
z_direct = 0
ax.quiver(x_pos, y_pos, z_pos, x_direct, y_direct, z_direct, length=2, arrow_length_ratio=0.3, color='black')
ax.text(x_pos, y_pos, z_pos, 'Earth LOS', size=10, zorder=1, color='black')

ax.grid(False)
extent = 4
ax.set_xlim(-extent, extent)
ax.set_ylim(-extent, extent)
ax.set_zlim(-extent, extent)
ax.set_box_aspect([extent, extent, extent])
ax.view_init(elev=90, azim=0)
ax.set_title(f"{m.meta['MAPDATE']} {m.meta['MAPTIME']} CR: {m.meta['CRCENTER']}")
for axis in 'xyz':
    getattr(ax, 'set_{}label'.format(axis))(axis+' [R$_\odot]$')
plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d', computed_zorder=False)

# draw the Sun
theta, phi = np.mgrid[0:np.pi:100j, 0:2*np.pi:100j]
x = np.sin(theta)*np.cos(phi) 
y = np.sin(theta)*np.sin(phi) 
z = np.cos(theta)
ax.plot_surface(x, y, z, color='orange', zorder=0)

# draw an arrow for the AR
latitude = 17
longitude = -72
x, y, z = geo_to_cartesian(latitude, longitude)
# ax.quiver(0, 0, 0, x, y, z, length=3, arrow_length_ratio=0.3, color='red', label='AR3680')

with tqdm(total=len(flines), desc='Plotting the field lines') as pbar:
    for n, field_line in enumerate(flines):
        color = {0:'black', -1:'blue', 1:'red'}.get(field_line.polarity)
        coords = field_line.coords
        coords.representation_type = 'cartesian'
        x_line = coords.x/const.radius
        y_line = coords.y/const.radius
        z_line = coords.z/const.radius
        ax.plot3D(x_line, y_line, z_line, alpha=0.3, color=color, linewidth=1, zorder=10)
        
        # if field_line.polarity==1 or field_line.polarity==-1:
        #     alpha = 0.3
        # else:
        #     alpha = 0
        # ax.plot3D(x_line, y_line, z_line, alpha=alpha, color=color, linewidth=1, zorder=10)
        
        pbar.update(1)

# draw Sun-Earth LOS direction
x_pos = 0
y_pos = -2
z_pos = 0
x_direct = 0
y_direct = -1
z_direct = 0
# ax.quiver(x_pos, y_pos, z_pos, x_direct, y_direct, z_direct, length=2, arrow_length_ratio=0.3, color='black', label='Earth LOS')

extent = 4
ax.set_xlim(-extent, extent)
ax.set_ylim(-extent, extent)
ax.set_zlim(-extent, extent)
ax.set_box_aspect([extent, extent, extent])
ax.view_init(elev=0, azim=180)
ax.grid(False)
ax.legend(loc='best')
ax.set_title(f"Azimuth: 180 deg. {m.meta['MAPDATE']} {m.meta['MAPTIME']} CR: {m.meta['CRCENTER']}")
for axis in 'xyz':
    getattr(ax, 'set_{}label'.format(axis))(axis+' [R$_\odot]$')
plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d', computed_zorder=False)

# draw the Sun
theta, phi = np.mgrid[0:np.pi:100j, 0:2*np.pi:100j]
x = np.sin(theta)*np.cos(phi) 
y = np.sin(theta)*np.sin(phi) 
z = np.cos(theta)
ax.plot_surface(x, y, z, color='orange', zorder=0)

# draw an arrow for the AR
latitude = 17
longitude = -72
x, y, z = geo_to_cartesian(latitude, longitude)
# ax.quiver(0, 0, 0, x, y, z, length=3, arrow_length_ratio=0.3, color='red', label='AR3680')

with tqdm(total=len(flines), desc='Plotting the field lines') as pbar:
    for n, field_line in enumerate(flines):
        color = {0:'black', -1:'blue', 1:'red'}.get(field_line.polarity)
        coords = field_line.coords
        coords.representation_type = 'cartesian'
        x_line = coords.x/const.radius
        y_line = coords.y/const.radius
        z_line = coords.z/const.radius
        ax.plot3D(x_line, y_line, z_line, alpha=0.3, color=color, linewidth=1, zorder=10)
        pbar.update(1)

# draw Sun-Earth LOS direction
x_pos = 0
y_pos = -2
z_pos = 0
x_direct = 0
y_direct = -1
z_direct = 0
# ax.quiver(x_pos, y_pos, z_pos, x_direct, y_direct, z_direct, length=2, arrow_length_ratio=0.3, color='black', label='Earth LOS')

extent = 4
ax.set_xlim(-extent, extent)
ax.set_ylim(-extent, extent)
ax.set_zlim(-extent, extent)
ax.set_box_aspect([extent, extent, extent])
ax.view_init(elev=0, azim=-90)
ax.grid(False)
ax.legend(loc='best')
ax.set_title(f"Azimuth: -90 deg. {m.meta['MAPDATE']} {m.meta['MAPTIME']} CR: {m.meta['CRCENTER']}")
for axis in 'xyz':
    getattr(ax, 'set_{}label'.format(axis))(axis+' [R$_\odot]$')
plt.tight_layout()
plt.show()

In [ ]:
1

In [ ]:
%matplotlib widget
# %matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Create a 3D plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d', computed_zorder=False)

# draw the Sun
theta, phi = np.mgrid[0:np.pi:100j, 0:2*np.pi:100j]
x = np.sin(theta)*np.cos(phi) 
y = np.sin(theta)*np.sin(phi) 
z = np.cos(theta)
ax.plot_surface(x, y, z, color='orange', zorder=0)

# Define your geographic coordinates
latitude = 0
longitude = -70
x, y, z = geo_to_cartesian(latitude, longitude)
ax.quiver(0, 0, 0, x, y, z, length=2, arrow_length_ratio=0.3, color='red')
ax.text(2, -2, 1, 'AR3680', size=10, zorder=10, color='red')

# draw Sun-Earth LOS direction
x_pos = 1
y_pos = 0
z_pos = -1.5
x_direct = 1
y_direct = 0
z_direct = 0
offset = 0.4
ax.text(x_pos, y_pos, z_pos, 'Earth LOS', size=10, zorder=1, color='black')
ax.quiver(x_pos, y_pos, z_pos, x_direct, y_direct, z_direct, length=2, arrow_length_ratio=0.3, color='black')

extent = 3
ax.set_xlim(-extent, extent)
ax.set_ylim(-extent, extent)
ax.set_zlim(-extent, extent)
ax.set_box_aspect([extent, extent, extent])
ax.grid(False)
for axis in 'xyz':
    getattr(ax, 'set_{}label'.format(axis))(axis+' [R$_\odot]$')
ax.view_init(elev=0, azim=0)
plt.tight_layout()
plt.show()

## Draw PFSS model on half Sun

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d', computed_zorder=False)

# draw half of the Sun
theta, phi = np.mgrid[0:np.pi:100j, 0:np.pi:100j]
x = np.sin(theta)*np.cos(phi) 
y = np.sin(theta)*np.sin(phi) 
z = np.cos(theta)

# applying rotation matrix to rotate the half of the Sun
Rx = np.array([[1, 0, 0],
               [0, np.cos(np.deg2rad(0)), -np.sin(np.deg2rad(0))],
               [0, np.sin(np.deg2rad(0)), np.cos(np.deg2rad(0))]])
x, y, z = np.einsum("ij,jkl->ikl", Rx, np.array([x, y, z]))
Rz = np.array([[np.cos(np.deg2rad(-90)), -np.sin(np.deg2rad(-90)), 0],
               [np.sin(np.deg2rad(-90)), np.cos(np.deg2rad(-90)), 0],
               [0, 0, 1]])
x, y, z = np.einsum("ij,jkl->ikl", Rz, np.array([x, y, z]))
ax.plot_surface(x, y, z, color='orange', zorder=0)

with tqdm(total=len(flines), desc='Plotting the field lines') as pbar:
    for n, field_line in enumerate(flines):
        color = {0:'black', -1:'blue', 1:'red'}.get(field_line.polarity)
        coords = field_line.coords
        coords.representation_type = 'cartesian'
        x_line = coords.x/const.radius
        y_line = coords.y/const.radius
        z_line = coords.z/const.radius
        ax.plot3D(x_line, y_line, z_line, alpha=0.4, color=color, linewidth=1, zorder=10)
        # if field_line.polarity==1 or field_line.polarity==-1:
        #     alpha_val = 0.4
        #     ax.plot3D(x_line, y_line, z_line, alpha=alpha_val, zorder=1, color=color, linewidth=1)
        # else:
        #     alpha_val = 0
        # ax.plot3D(x_line, y_line, z_line, alpha=alpha_val, zorder=1, color=color, linewidth=1)
        pbar.update(1)

# draw Sun-Earth LOS direction
x_pos = 2
y_pos = 0
z_pos = 0
x_direct = 1
y_direct = 0
z_direct = 0
ax.quiver(x_pos, y_pos, z_pos, x_direct, y_direct, z_direct, length=2, arrow_length_ratio=0.3, color='black')
ax.text(x_pos, y_pos, z_pos, 'Earth LOS', size=10, zorder=1, color='black')

# plot settings 
ax.grid(False)
ax.set_xlim(0, 4)
ax.set_ylim(-3, 3)
ax.set_zlim(-3, 3)
ax.set_box_aspect([2, 3, 3])
#ax.view_init(elev=0, azim=0)
for axis in 'xyz':
    getattr(ax, 'set_{}label'.format(axis))(axis+' [R$_\odot]$')
plt.tight_layout()
plt.show()

In [ ]:
num_footpoints = 20
lat = np.linspace(-0.4*np.pi, 0.5*np.pi, num_footpoints, endpoint=False)
lon = np.linspace(-0.5*np.pi, 1.5*np.pi, num_footpoints+10, endpoint=False)

lat, lon = np.meshgrid(lat, lon, indexing='ij')
lat, lon = lat.ravel()*u.rad, lon.ravel()*u.rad
r = 1.2*const.radius

# Make a 2D grid from these 1D points 
seeds  = SkyCoord(lon, lat, r, frame=pfss_out.coordinate_frame)

In [ ]:
tracer = tracing.FortranTracer()
flines = tracer.trace(seeds, pfss_out)

In [ ]:
fig = plt.figure()
ax = plt.subplot(111, projection=m_c2_rr)
m_c2_rr.plot(axes=ax)

with tqdm(total=len(flines), desc='Plotting the field lines') as pbar:
    for fline in flines:
        color = {0:'black', -1:'blue', 1:'red'}.get(fline.polarity)
        ax.plot_coord(fline.coords, alpha=0.5, color=color, linewidth=1)
        pbar.update(1)

ax.set_xlim(left=0, right=int(m_c2_rr.dimensions.x.value))
ax.set_ylim(bottom=0, top=int(m_c2_rr.dimensions.y.value))
fig.tight_layout()
fig.savefig(f'{data_dir}/LASCO_C2_runratio_PFSS.pdf', format='pdf', bbox_inches='tight')
# fig.savefig(f'{data_dir}/LASCO_C2_runratio_PFSS.png', format='png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

theta, phi = np.mgrid[0:np.pi:100j, 0:2*np.pi:100j]
x = np.sin(theta)*np.cos(phi) 
y = np.sin(theta)*np.sin(phi) 
z = np.cos(theta)
ax.plot_surface(x, y, z, color='orange', zorder=0)

with tqdm(total=len(flines), desc='Plotting the field lines') as pbar:
    for n, field_line in enumerate(flines):
        color = {0:'black', -1:'blue', 1:'red'}.get(field_line.polarity)
        coords = field_line.coords
        coords.representation_type = 'cartesian'
        x_line = coords.x/const.radius
        y_line = coords.y/const.radius
        z_line = coords.z/const.radius
        
        ax.plot3D(x_line, y_line, z_line, alpha=0.4, color=color, linewidth=1, zorder=10)
        
        # if field_line.polarity==1 or field_line.polarity==-1:
        #     alpha = 0.3
        # else:
        #     alpha = 0
        # ax.plot3D(x_line, y_line, z_line, alpha=alpha, color=color, linewidth=1, zorder=10)
        
        pbar.update(1)

# draw Sun-Earth LOS direction
x_pos = 3
y_pos = 0
z_pos = 0
x_direct = 1
y_direct = 0
z_direct = 0
ax.quiver(x_pos, y_pos, z_pos, x_direct, y_direct, z_direct, length=2, arrow_length_ratio=0.3, color='black')
ax.text(x_pos, y_pos, z_pos, 'Earth LOS', size=10, zorder=1, color='black')

ax.grid(False)
extent = 4
ax.set_xlim(-extent, extent)
ax.set_ylim(-extent, extent)
ax.set_zlim(-extent, extent)
ax.set_box_aspect([extent, extent, extent])
ax.view_init(elev=90, azim=0)
for axis in 'xyz':
    getattr(ax, 'set_{}label'.format(axis))(axis+' [R$_\odot]$')
plt.tight_layout()
plt.show()

In [ ]:
# The PFSS solution is calculated on a regular 3D grid in (phi, s, rho), where rho = ln(r), 
# and r is the standard spherical radial coordinate.
# We need to define the number of grid points in rho, and the source surface radius.
# From the boundary condition, number of radial grid points, and source surface, 
# we now construct an `Input` object that stores this information.
# Using the `Input` object, plot the input photospheric magnetic field.
rss = 7
nrho = 50
pfss_in = pfsspy.Input(gong_map, nrho, rss)
pfss_out = pfsspy.pfss(pfss_in)

# Now we construct a lat x lon grid of footpoitns to trace some magnetic field lines from. 
# These coordinates are defined in the helioprojective frame of the AIA image.
num_footpoints = 30
lat = np.linspace(np.radians(-89), np.radians(89), num_footpoints+15, endpoint=False)
lon = np.linspace(np.radians(180), np.radians(270), num_footpoints, endpoint=False)

lat, lon = np.meshgrid(lat, lon, indexing='ij')
lat, lon = lat.ravel()*u.rad, lon.ravel()*u.rad
seedheight = 1.2*const.radius # The height at which field line tracing is started, in solar radii

# Make a 2D grid from these 1D points
seeds = SkyCoord(lon, lat, seedheight, frame=pfss_out.coordinate_frame)
tracer = tracing.FortranTracer(max_steps=20000, step_size=1)
# tracer = tracing.PythonTracer()
flines = tracer.trace(seeds, pfss_out)

In [ ]:
# Plot the magnetogram and the seed footpoints
m = pfss_in.map

fig = plt.figure(figsize=[10,5])
ax = plt.subplot(projection=m)
gon = m.plot()
ax.plot_coord(seeds, color='black', marker='o', linewidth=0, markersize=2)
fig.colorbar(gon, shrink=0.8, pad=0.03)
fig.tight_layout()
plt.show()

In [ ]:
fig = plt.figure()
ax = plt.subplot(111, projection=suvi_map)
suvi_map.plot(axes=ax, cmap='gray_r')
ax.plot_coord(seeds, color='dodgerblue', marker='o', markersize=3, linewidth=0)

# for fline in flines:
#     ax.plot_coord(fline.coords, alpha=0.3, color='red', linewidth=1)

# ax.set_xlabel('Helioprojective Longitude [arcsec]')
# ax.set_ylabel('Helioprojective Latitutde [arcsec]')
# ax.set_xlim(left=0, right=int(suvi_map.dimensions.x.value))
# ax.set_ylim(bottom=0, top=int(suvi_map.dimensions.y.value))

fig.tight_layout()
plt.show()

### Or .. show the seeds first on SUVI then on GONG ...

In [ ]:
hp_lon = np.linspace(-970, -700, 5) * u.arcsec
hp_lat = np.linspace(-970, 970, 25) * u.arcsec

# Make a 2D grid from these 1D points
lon, lat = np.meshgrid(hp_lon, hp_lat)
seeds = SkyCoord(lon.ravel(), lat.ravel(), frame=suvi_map.coordinate_frame)

fig = plt.figure()
ax = plt.subplot(projection=suvi_map)
suvi_map.plot(axes=ax, cmap='gray_r')
ax.plot_coord(seeds, color='crimson', marker='o', markersize=3, linewidth=0)
fig.tight_layout()
plt.show()

In [ ]:
# Plot the magnetogram and the seed footpoints
m = pfss_in.map

fig = plt.figure(figsize=[10,5])
ax = plt.subplot(projection=m)
gon = m.plot()
ax.plot_coord(seeds, color='black', marker='o', linewidth=0, markersize=2)
fig.colorbar(gon, shrink=0.8, pad=0.03)
fig.tight_layout()
plt.show()

In [ ]:
### TEST to find where is the Earth LOS on GONG map ...
hp_lon = np.linspace(-5, 5, 3) * u.arcsec
hp_lat = np.linspace(-970, 970, 25) * u.arcsec

# Make a 2D grid from these 1D points
lon, lat = np.meshgrid(hp_lon, hp_lat)
seeds = SkyCoord(lon.ravel(), lat.ravel(), frame=suvi_map.coordinate_frame)

fig = plt.figure()
ax = plt.subplot(projection=suvi_map)
suvi_map.plot(axes=ax, cmap='gray_r')
ax.plot_coord(seeds, color='crimson', marker='o', markersize=3, linewidth=0)
fig.tight_layout()
plt.show()

# Plot the magnetogram and the seed footpoints
m = pfss_in.map

fig = plt.figure(figsize=[10,5])
ax = plt.subplot(projection=m)
gon = m.plot()
ax.plot_coord(seeds, color='black', marker='o', linewidth=0, markersize=2)
fig.colorbar(gon, shrink=0.8, pad=0.03)
fig.tight_layout()
plt.show()

In [ ]:
sunpy.coordinates.sun.L0(time=suvi_map.meta['date-obs'])

In [ ]:
print(sunpy.coordinates.sun.L0(time=suvi_map.meta['date-obs']).value)

In [ ]:
# Create a 3D plot
fig = plt.figure()
ax = fig.add_subplot(projection='3d', computed_zorder=False)

# draw the Sun
theta, phi = np.mgrid[0:np.pi:100j, 0:2*np.pi:100j]
x = np.sin(theta)*np.cos(phi) 
y = np.sin(theta)*np.sin(phi) 
z = np.cos(theta)
ax.plot_surface(x, y, z, color='orange', zorder=0)

# Define your geographic coordinates
latitude = 0
longitude = sunpy.coordinates.sun.L0(time=suvi_map.meta['date-obs'])
x, y, z = geo_to_cartesian(latitude, longitude)
ax.quiver(0, 0, 0, x, y, z, length=2, arrow_length_ratio=0.3, color='red', label='Correct Earth LOS?')

# x_direct = 1
# y_direct = 0
# z_direct = 0
# ax.quiver(1, 0, 0, x_direct, y_direct, z_direct, length=2, arrow_length_ratio=0.3, color='black', label='Earth LOS')

extent = 3
ax.set_xlim(-extent, extent)
ax.set_ylim(-extent, extent)
ax.set_zlim(-extent, extent)
ax.set_box_aspect([extent, extent, extent])
ax.grid(False)
ax.legend(loc='best')
for axis in 'xyz':
    getattr(ax, 'set_{}label'.format(axis))(axis+' [R$_\odot]$')
ax.view_init(elev=90, azim=sunpy.coordinates.sun.L0(time=suvi_map.meta['date-obs']))
fig.tight_layout()
plt.show()